In [16]:
import glob
import pandas as pd
import os
import torch
import librosa
import random
import torch.nn as nn

from BEATs.Tokenizers import TokenizersConfig, Tokenizers
from BEATs.BEATs import BEATs, BEATsConfig

In [ ]:
data_folder = "/data/ESC-50-master"
audio = glob.glob(data_folder + "/audio/*.wav", recursive=True)
labels = pd.read_csv(os.path.join(data_folder, "meta/esc50.csv"))

In [ ]:
df_audio = pd.DataFrame(audio, columns=["filepath"])
df_audio["filename"] = [f.split("/")[-1] for f in audio]
filepath_labels = labels.merge(df_audio, how="inner", on="filename")

In [63]:
# load the pre-trained checkpoints
checkpoint = torch.load('/data/BEATs/BEATs_iter3_plus_AS2M.pt')
cfg = BEATsConfig(
            {
                **checkpoint["cfg"]
            }
        )
BEATs_model = BEATs(cfg)

ft = torch.load("/app/lightning_logs/version_0/checkpoints/epoch=9-step=500.ckpt")
ft["state_dict"] = {key.replace('beats.', ''): value for key, value in ft["state_dict"].items() if not key.endswith(('fc.weight', 'fc.bias'))}
BEATs_model.load_state_dict(ft["state_dict"])

<All keys matched successfully>